In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from solar import REF_SOLAR_DATA
from batteryopt import process_pge_meterdata, merge_solar_and_load_data, build_tariff, run_optimization

%load_ext autoreload
%autoreload 2

In [ ]:
solar_size_kw = 1.0
batt_size_kwh = 13.5
csv_file = 'data/pge-e78ff14c-c8c0-11ec-8cc7-0200170a3297-DailyUsageData/pge_electric_usage_interval_data_Service 1_1_2024-02-01_to_2025-01-31.csv'

elec_usage = process_pge_meterdata(csv_file)
site_data = merge_solar_and_load_data(elec_usage, solar_size_kw * REF_SOLAR_DATA)
tariff = build_tariff(site_data.index)
battery_dispatch = run_optimization(site_data, tariff, batt_e_max=batt_size_kwh)
all_input = pd.concat([site_data, tariff, battery_dispatch], axis=1)

final_week = all_input.loc[all_input.index[-1] - pd.DateOffset(days=7):]

fig, ax = plt.subplots()
final_week.plot(ax=ax)

In [ ]:
tmp_output = all_input[['load', 'solar', 'px_buy', 'px_sell']]
# tmp_output.to_csv('data/tmp_out_full.csv', index=False)
# tmp_output.loc[all_input.index[-1] - pd.DateOffset(days=2):].to_csv('data/tmp_out_noindex.csv', index=False)


In [1]:
import math
import pandas as pd
import pulp as pl

def build_and_export_pulp(input_df: pd.DataFrame,
                          batt_rt_eff=0.85,
                          batt_e_max=13.5,
                          batt_p_max=5,
                          dt=1.0,
                          filename="model_pulp.mps"):
    input_df = input_df[['load', 'solar', 'px_buy', 'px_sell']]
    T = list(input_df.index)  # time steps
    
    # Parameters
    oneway_eff = math.sqrt(batt_rt_eff)
    inv_oneway_eff = 1.0 / oneway_eff
    backup_reserve = 0.2
    e_min = backup_reserve * batt_e_max
    E_0 = e_min

    # Problem
    prob = pl.LpProblem("BatteryGrid", pl.LpMinimize)

    # Variables
    P_charge   = pl.LpVariable.dicts("P_batt_charge",   T, lowBound=-batt_p_max, upBound=0)
    P_discharge= pl.LpVariable.dicts("P_batt_discharge",T, lowBound=0,        upBound=batt_p_max)
    P_buy      = pl.LpVariable.dicts("P_grid_buy",      T, lowBound=0)
    P_sell     = pl.LpVariable.dicts("P_grid_sell",     T, lowBound=None,     upBound=0)
    E_state    = pl.LpVariable.dicts("E",                list(range(len(T)+1)),
                                     lowBound=e_min,     upBound=batt_e_max)

    # Objective
    prob += pl.lpSum([
        P_sell[t] * input_df.loc[t, "px_sell"]
      + P_buy[t]  * input_df.loc[t, "px_buy"]
      for t in T
    ])

    # Constraints
    # Initial SOC
    prob += E_state[0] == E_0

    # Energy dynamics & power balance at each step
    for i, t in enumerate(T):
        # E[i+1] = E[i] - (charge*eff + discharge/eff)*dt
        prob += ( E_state[i+1]
                == E_state[i]
                   - (P_charge[t] * oneway_eff
                      + P_discharge[t] * inv_oneway_eff) * dt )
        # Power balance: charge + discharge + buy + sell + net load = 0
        prob += ( P_charge[t]
                + P_discharge[t]
                + P_buy[t]
                + P_sell[t]
                - input_df.loc[t, "load"]
                + input_df.loc[t, "solar"]
                == 0 )

    # Export to MPS
    prob.writeMPS(filename)
    print(f"Wrote PuLP MPS to {filename}")

In [2]:
import pathlib

data_root = pathlib.Path('data')
output_root = pathlib.Path('mps_files')


input_fnames = [
    'tmp_full_noindex.csv',
    'tmp_half_noindex.csv',
    'tmp_quarter_noindex.csv',
    'tmp_fivehundred_rows_noindex.csv',
    'tmp_onethousand_noindex.csv',
]

for f in input_fnames:
    input_f = data_root / f
    output_f = output_root / (input_f.stem + '.mps')
    input_df = pd.read_csv(input_f)
    build_and_export_pulp(input_df, filename=output_f)

Wrote PuLP MPS to mps_files/tmp_full_noindex.mps
Wrote PuLP MPS to mps_files/tmp_half_noindex.mps
Wrote PuLP MPS to mps_files/tmp_quarter_noindex.mps
Wrote PuLP MPS to mps_files/tmp_fivehundred_rows_noindex.mps
Wrote PuLP MPS to mps_files/tmp_onethousand_noindex.mps


# Experimental runtimes

tmp_full_noindex.mps 0.706
tmp_half_noindex.mps 0.390
tmp_quarter_noindex.mps 0.276
tmp_onethousand_noindex.mps 0.063
tmp_fivehundred_rows_noindex.mps 0.026